In [4]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing import sequence

In [5]:
import pandas as pd
import numpy as np
import re

In [6]:
# bring in matching records for evaluation of the Keras_1D classifier 
df = pd.read_csv('/Users/mh302/OneDrive - West Point/Backup/ORCEN/CPO/record_linkage/obj2_recs.csv', encoding='utf-8')
df[['MaintOperationDetailDesc', 'CorroPerc', 'DESCRIPTION_x']]

,MaintOperationDetailDesc,CorroPerc,DESCRIPTION_x
0,COMPLETED,0.40,117-Deteriorated
1,COMPLETED,0.40,117-Deteriorated
2,COMPLETED. REPLACED IN CONJUNCTION WITH AXIAL ...,0.00,170-Corroded
3,COMPLETED,0.00,117-Deteriorated
4,REINSTALLED INTERMEDIATE GEARBOX ACCELEROMETER...,0.33,117-Deteriorated
...,...,...,...
490,REINSTALLED,0.00,170-Corroded
491,RECONNECTED 4EA ELECTRICAL CONNECTORS TO DEICE...,0.00,170-Corroded
492,COMPLETED OK,0.05,170-Corroded
493,INSPECTION COMPLETE CHECKS FOUND OK,0.33,170-Corroded


In [7]:
# make the y_MADW column an indicator for 1 = corrosion-tag, 0 = no corrosion
y = []
for row in df.DESCRIPTION_x:
    if row in ['170-Corroded', '520-Pitted', '050-Blistered', '240-Flaking']:
        y.append(1)
    else:
        y.append(0)
        
# make the y_MADW column an indicator for 1 = corrosion-tag, 0 = no corrosion
y_MADW = []
for row in df.CorroPerc:
    if row !=0:
        y_MADW.append(1)
    else:
        y_MADW.append(0)
        
print(sum(y))
print(sum(y_MADW))

321
182


In [8]:
df['y'] = y
df['y_MADW'] = y_MADW

In [9]:
# Preprocess text just like I did for training the convnet

# Ensure that text variables are strings
df.MaintOperationDetailDesc = df.MaintOperationDetailDesc.astype(str)


# Remove special characters
def clean_text(x):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', x)
    return text

cleaned_text = []
for text in df['MaintOperationDetailDesc']:
    cleaned_text.append(clean_text(text.lower()))

# Replace contractions
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

# Usage
#replace_contractions("this's a text with contraction")

contr_cleaned_text = []
for text in cleaned_text:
    contr_cleaned_text.append(replace_contractions(text))

# remove numbers
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

num_contr_cleaned_text = []
for text in contr_cleaned_text:
    num_contr_cleaned_text.append(clean_numbers(text))

#num_contr_cleaned_text[0:5]

from nltk.corpus import stopwords

x_no_stp_wds = []
for doc in num_contr_cleaned_text:
    filtered_words = []
    for word in doc.split():
        if word.lower() not in stopwords.words('english'):
            filtered_words.append(word.lower())
        else:
            next
    x_no_stp_wds.append(' '.join(filtered_words)) #joins the words into a sentence, appends the sentence as list to x_no_stp_wds

# from nltk.corpus import stopwords
# from joblib import Parallel, delayed
# import multiprocessing

# def remove_stp_wds(doc, stp_wds= stopwords.words('english')):
#     filtered_words = []
#     for word in doc.split():
#         if word.lower() not in stp_wds:
#             filtered_words.append(word.lower())
#         else:
#             next
#     return(' '.join(filtered_words)) 

# num_cores = multiprocessing.cpu_count()
# x_no_stp_wds = Parallel(n_jobs=num_cores)(delayed(remove_stp_wds)(doc) for doc in num_contr_cleaned_text)

x_no_stp_wds[0:10]

['completed',
 'completed',
 'completed replaced conjunction axial fan replacement',
 'completed',
 'reinstalled intermediate gearbox accelerometers brackets',
 'carried ##### event #### dated 5 june ####',
 'check complete',
 'comp passed ck iaw em #### wp #### ## mr ##### tq ##### lbs',
 'complete',
 'completed']

In [10]:
from keras.preprocessing.text import Tokenizer

# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(x_no_stp_wds)
# summarize what was learned
#Once fit, the Tokenizer provides 4 attributes that you can use to query what has been learned about your documents:

#word_counts: A dictionary of words and their counts.
#word_docs: A dictionary of words and how many documents each appeared in.
#word_index: A dictionary of words and their uniquely assigned integers.
#document_count:An integer count of the total number of documents that were used to fit the Tokenizer
#print(t.word_counts)
print(t.document_count) # total number of docs
len(t.word_index) # total number of words in the vocab (originally 94819 without removing stopwords)
#print(t.word_docs) 

495


313

In [11]:
# set parameters:
max_features = 10000 # cut down the vocab size
maxlen = 400 # cut down the max document length

# integer encode the documents
docs = x_no_stp_wds
labels = df['y']

encoded_docs = [one_hot(d, max_features) for d in docs]

In [12]:
x_test = encoded_docs
y_test = labels
print(len(x_test), 'test sequences')


495 test sequences


In [13]:
print('Pad sequences (samples x time)')

x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

print('x_test shape:', x_test.shape)



Pad sequences (samples x time)
x_test shape: (495, 400)


## 1DConv Model

In [16]:
from keras.models import load_model
model = load_model('/Users/mh302/OneDrive - West Point/Backup/ORCEN/CPO/deep_learning/corr_model_1.h5')

/Users/mh302/opt/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
corr_model_1DConv_pretrnd_fixed

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [18]:
# https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/ 
# predict probabilities for test set
yhat_probs = model.predict(x_test, verbose=0)
# predict crisp classes for test set
yhat_classes = model.predict_classes(x_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]
 
# accuracy: (tp + tn) / (p + n), the ratio of correctly predicted observation to the total observations.
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp), the ratio of correctly predicted positive observations to the total predicted positive observations
# If precision is high, we are making correct corrosion tags, but possibly missing corrosion items
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn), ratio of correctly predicted positive observations to the all observations in actual class
# So if recall is low, we are missing actual corrosion actions
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn), the weighted average of Precision and Recall
# If this is high, we are making correct corrosion tags and not missing actions that should be corrosion.
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

Accuracy: 0.402020
Precision: 0.564767
Recall: 0.339564
F1 score: 0.424125


In [25]:
df['yhat_classes'] = yhat_classes
df[['y', 'yhat_classes', 'y_MADW', 'MaintOperationDetailDesc']][yhat_classes!=y_MADW].to_csv('error_analysis.csv', index=True)
df[['y', 'yhat_classes', 'y_MADW', 'MaintOperationDetailDesc']][yhat_classes!=y_MADW]

,y,yhat_classes,y_MADW,MaintOperationDetailDesc
2,1,1,0,COMPLETED. REPLACED IN CONJUNCTION WITH AXIAL ...
3,0,1,0,COMPLETED
6,1,0,1,CHECK COMPLETE
7,1,0,1,COMP PASSED CK IAW EM 0284 WP 1556 00 M/R 1264...
10,0,0,1,INSPECTION FOUND OK TQ S/N 5595
...,...,...,...,...
478,1,1,0,REPLACED. TQ = 250 FT. LBS. S/N: B1230 (18 APR...
479,0,1,0,RECONNECTED
484,1,1,0,"N/A, NBIT"
486,0,0,1,COMPLETE IAW EM0013 WP 0572 00


Of  495  matching records from MADW to ASAP-RAM "matches"  
Proportion of "matching" records with a corr match: 0.416  
MADW Expert System Performance:  
Accuracy: 0.416162  
Precision: 0.587912  
Recall: 0.333333  
F1 score: 0.425447  

In [20]:
num_correct = len(y_test[y_test == yhat_classes])
nobs = len(y_test)
print(num_correct/nobs)
from statsmodels.stats import proportion as prop
# compute 95% CI on model accuracy
match_prop = prop.proportion_confint(count=num_correct, nobs=nobs, alpha=0.05, method='normal')
match_prop

0.402020202020202


(0.3588272699450845, 0.4452131340953195)

## 1DConv Model Pretrained Trainable Embeddings

In [28]:
from keras.models import load_model
model = load_model('/Users/mh302/OneDrive - West Point/Backup/ORCEN/CPO/deep_learning/corr_model_1DConv_pretrnd.h5')

/Users/mh302/opt/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [29]:
# https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/ 
# predict probabilities for test set
yhat_probs = model.predict(x_test, verbose=0)
# predict crisp classes for test set
yhat_classes = model.predict_classes(x_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]
 
# accuracy: (tp + tn) / (p + n), the ratio of correctly predicted observation to the total observations.
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp), the ratio of correctly predicted positive observations to the total predicted positive observations
# If precision is high, we are making correct corrosion tags, but possibly missing corrosion items
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn), ratio of correctly predicted positive observations to the all observations in actual class
# So if recall is low, we are missing actual corrosion actions
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn), the weighted average of Precision and Recall
# If this is high, we are making correct corrosion tags and not missing actions that should be corrosion.
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

Accuracy: 0.648485
Precision: 0.648485
Recall: 1.000000
F1 score: 0.786765


Of  495  matching records from MADW to ASAP-RAM "matches"  
Proportion of "matching" records with a corr match: 0.416  
MADW Expert System Performance:  
Accuracy: 0.416162  
Precision: 0.587912  
Recall: 0.333333  
F1 score: 0.425447  

In [26]:
# improvement on F1 Score
print("DL model improvement on F1 by: ", abs(0.425447 - 0.7868) / 0.425447 * 100, "%")

DL model improvement on F1 by:  84.93490376004532 %


In [27]:
# improvement on accuracy
print("DL model improvement on accuracy by: ", abs( 0.416162 - 0.6485) /  0.416162 * 100, "%")

DL model improvement on accuracy by:  55.82873976960895 %


In [25]:
num_correct = len(y_test[y_test == yhat_classes])
nobs = len(y_test)
print(num_correct/nobs)
from statsmodels.stats import proportion as prop
# compute 95% CI on model accuracy
match_prop = prop.proportion_confint(count=num_correct, nobs=nobs, alpha=0.05, method='normal')
match_prop

0.6484848484848484


(0.6064250262356793, 0.6905446707340176)

### The 1D Conv Nets with pretrained embeddings (both trainable and un-trainable) resulted in the same model!  This means that the pre-trained embeddings were optimal for the 1D Convnet task (doc classification).  Very interesting.  

## 1DConv Model Pretrained Fixed Embeddings

In [23]:
from keras.models import load_model
model = load_model('/Users/mh302/OneDrive - West Point/Backup/ORCEN/CPO/deep_learning/corr_model_1DConv_pretrnd_fixed.h5')

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [24]:
# https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/ 
# predict probabilities for test set
yhat_probs = model.predict(x_test, verbose=0)
# predict crisp classes for test set
yhat_classes = model.predict_classes(x_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]
 
# accuracy: (tp + tn) / (p + n), the ratio of correctly predicted observation to the total observations.
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp), the ratio of correctly predicted positive observations to the total predicted positive observations
# If precision is high, we are making correct corrosion tags, but possibly missing corrosion items
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn), ratio of correctly predicted positive observations to the all observations in actual class
# So if recall is low, we are missing actual corrosion actions
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn), the weighted average of Precision and Recall
# If this is high, we are making correct corrosion tags and not missing actions that should be corrosion.
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

Accuracy: 0.648485
Precision: 0.648485
Recall: 1.000000
F1 score: 0.786765


Of  495  matching records from MADW to ASAP-RAM "matches"  
Proportion of "matching" records with a corr match: 0.416  
MADW Expert System Performance:  
Accuracy: 0.416162  
Precision: 0.587912  
Recall: 0.333333  
F1 score: 0.425447  

In [26]:
# improvement on F1 Score
print("DL model improvement on F1 by: ", abs(0.425447 - 0.7868) / 0.425447 * 100, "%")

DL model improvement on F1 by:  84.93490376004532 %


In [30]:
# improvement on accuracy
print("DL model improvement on accuracy by: ", abs( 0.416162 - 0.6485) /  0.416162 * 100, "%")

# improvement on recall
print("DL model improvement on accuracy by: ", abs( 0.333 - 1) /  0.333 * 100, "%")

DL model improvement on accuracy by:  55.82873976960895 %
DL model improvement on accuracy by:  200.30030030030028 %


In [25]:
num_correct = len(y_test[y_test == yhat_classes])
nobs = len(y_test)
print(num_correct/nobs)
from statsmodels.stats import proportion as prop
# compute 95% CI on model accuracy
match_prop = prop.proportion_confint(count=num_correct, nobs=nobs, alpha=0.05, method='normal')
match_prop

0.6484848484848484


(0.6064250262356793, 0.6905446707340176)

## LSTM Model Testing

In [26]:
from keras.models import load_model
model_lstm = load_model('/Users/mh302/OneDrive - West Point/Backup/ORCEN/CPO/deep_learning/atn_model_norm_corpus_unique.h5')

/Users/mh302/opt/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [29]:
# https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/ 
# predict probabilities for test set
yhat_probs = model_lstm.predict(x_test, verbose=0)
# predict crisp classes for test set
yhat_classes = [round(pred[0]) for pred in yhat_probs]
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]

# accuracy: (tp + tn) / (p + n), the ratio of correctly predicted observation to the total observations.
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp), the ratio of correctly predicted positive observations to the total predicted positive observations
# If precision is high, we are making correct corrosion tags, but possibly missing corrosion items
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn), ratio of correctly predicted positive observations to the all observations in actual class
# So if recall is low, we are missing actual corrosion actions
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn), the weighted average of Precision and Recall
# If this is high, we are making correct corrosion tags and not missing actions that should be corrosion.
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

Accuracy: 0.549495
Precision: 0.745000
Recall: 0.464174
F1 score: 0.571977


In [32]:
num_correct = len(y_test[y_test == yhat_classes])
nobs = len(y_test)
print(num_correct/nobs)
from statsmodels.stats import proportion as prop
# compute 95% CI on model accuracy
match_prop = prop.proportion_confint(count=num_correct, nobs=nobs, alpha=0.05, method='normal')
match_prop

0.5494949494949495


(0.5056643736782512, 0.5933255253116477)

In [33]:
# improvement on F1 Score
print("DL model improvement on F1 by: ", abs(0.425447 - 0.571977) / 0.425447 * 100, "%")

DL model improvement on F1 by:  34.44142278591691 %


In [34]:
# improvement on accuracy
print("DL model improvement on accuracy by: ", abs( 0.416162 - 0.549495) /  0.416162 * 100, "%")

DL model improvement on accuracy by:  32.03872530408831 %


## LSTM Model with trainable pre-trained embeddings

In [1]:
from keras.models import load_model
model_lstm = load_model('/Users/mh302/OneDrive - West Point/Backup/ORCEN/CPO/deep_learning/atn_model_norm_corpus_unique_pretrn_emb.h5')

Using TensorFlow backend.
/Users/mh302/opt/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [2]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [14]:
# https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/ 
# predict probabilities for test set
yhat_probs = model_lstm.predict(x_test, verbose=0)
# predict crisp classes for test set
yhat_classes = [round(pred[0]) for pred in yhat_probs]
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]

# accuracy: (tp + tn) / (p + n), the ratio of correctly predicted observation to the total observations.
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp), the ratio of correctly predicted positive observations to the total predicted positive observations
# If precision is high, we are making correct corrosion tags, but possibly missing corrosion items
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn), ratio of correctly predicted positive observations to the all observations in actual class
# So if recall is low, we are missing actual corrosion actions
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn), the weighted average of Precision and Recall
# If this is high, we are making correct corrosion tags and not missing actions that should be corrosion.
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

Accuracy: 0.444444
Precision: 0.632184
Recall: 0.342679
F1 score: 0.444444


In [15]:
num_correct = len(y_test[y_test == yhat_classes])
nobs = len(y_test)
print(num_correct/nobs)
from statsmodels.stats import proportion as prop
# compute 95% CI on model accuracy
match_prop = prop.proportion_confint(count=num_correct, nobs=nobs, alpha=0.05, method='normal')
match_prop

0.4444444444444444


(0.40067026842202497, 0.4882186204668639)

In [17]:
# improvement on F1 Score
print("DL model improvement on F1 by: ", abs(0.425447 - 0.4444444) / 0.425447 * 100, "%")

DL model improvement on F1 by:  4.465280046633305 %


In [18]:
# improvement on accuracy
print("DL model improvement on accuracy by: ", abs( 0.416162 - 0.4444444) /  0.416162 * 100, "%")

DL model improvement on accuracy by:  6.796007324070925 %


### Apparently, using pre-trained embeddings makes the LSTM perform more similarly to the expert system model.  ie we are making the DL model use the human (expert) understanding of words vs allowing the machine to learn from experience on the matinenance data. 

## LSTM Model with fixed pre-trained embeddings

In [19]:
from keras.models import load_model
model_lstm = load_model('/Users/mh302/OneDrive - West Point/Backup/ORCEN/CPO/deep_learning/atn_model_norm_corpus_unique_pretrn_emb_fixed.h5')

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [20]:
# https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/ 
# predict probabilities for test set
yhat_probs = model_lstm.predict(x_test, verbose=0)
# predict crisp classes for test set
yhat_classes = [round(pred[0]) for pred in yhat_probs]
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]

# accuracy: (tp + tn) / (p + n), the ratio of correctly predicted observation to the total observations.
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp), the ratio of correctly predicted positive observations to the total predicted positive observations
# If precision is high, we are making correct corrosion tags, but possibly missing corrosion items
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn), ratio of correctly predicted positive observations to the all observations in actual class
# So if recall is low, we are missing actual corrosion actions
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn), the weighted average of Precision and Recall
# If this is high, we are making correct corrosion tags and not missing actions that should be corrosion.
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

Accuracy: 0.658586
Precision: 0.674312
Recall: 0.915888
F1 score: 0.776750


In [15]:
num_correct = len(y_test[y_test == yhat_classes])
nobs = len(y_test)
print(num_correct/nobs)
from statsmodels.stats import proportion as prop
# compute 95% CI on model accuracy
match_prop = prop.proportion_confint(count=num_correct, nobs=nobs, alpha=0.05, method='normal')
match_prop

0.4444444444444444


(0.40067026842202497, 0.4882186204668639)

In [21]:
# improvement on F1 Score
print("DL model improvement on F1 by: ", abs(0.425447 - 0.77675) / 0.425447 * 100, "%")

DL model improvement on F1 by:  82.57268237876869 %


In [22]:
# improvement on accuracy
print("DL model improvement on accuracy by: ", abs( 0.416162 - 0.6586) /  0.416162 * 100, "%")

DL model improvement on accuracy by:  58.25567927874241 %


### Fixing the pre-trained embeddings made the model a poorer performer on the train/val set, but big improvement on the ASAP-RAM test set! 